In [ ]:
# Import relevant libraries
import os
import pandas as pd
from ModularCirc.Models.NaghaviModel import NaghaviModel, NaghaviModelParameters, TEMPLATE_TIME_SETUP_DICT
from SALib.sample import saltelli
import json
from ModularCirc import BatchRunner
from comparative_gsa.simulate_data import simulate_data
from comparative_gsa.calculate_output_features import calculate_output_features
from SALib.analyze.sobol import analyze

In [ ]:
param_path = '../inputs/parameters_naghavi_constrained_fixed_T_v_tot.json'
# Get the filename from the path, without extension
param_filename = os.path.splitext(os.path.basename(param_path))[0]

In [ ]:
# Set up the batch runner

## read in save parameters to the folder where simulations r saved.
with open(param_path, 'r') as f:
    params = json.load(f)

# Set up the batch runner
br = BatchRunner('Sobol', 0) # why are we using 'Sobol' here?
br.setup_sampler(param_path)
br.sample(10**6) # setting to a large number just to have more than n_samples. It will later get truncated down to n_samples

In [ ]:
br.samples

In [ ]:
br._parameters_2_sample

In [ ]:
# Now, generate samples for sensitivity analysis by taking the keys of _parameters_2_sample and turning into a list
relevant_columns = list(br._parameters_2_sample.keys())
relevant_columns

In [ ]:
problem = {
    'num_vars': len(relevant_columns),
    'names': relevant_columns,
    'bounds' : br.samples[relevant_columns].describe().loc[['min', 'max']].T.values
}

In [ ]:
param_values = saltelli.sample(problem, 16, calc_second_order=True)
param_values.shape

In [ ]:
param_values = pd.DataFrame(param_values) 

param_values.columns = relevant_columns

param_values

In [ ]:
# How many rows are in param_values
param_values.shape[0]

# Truncate br.samples to be only the first n_samples rows
br._samples = br._samples.iloc[:param_values.shape[0]].copy()

In [ ]:
br.samples

In [ ]:
n_samples = param_values.shape[0]

simulation_out_path = f'../outputs/simulations/output_{n_samples}_samples_{param_filename}/'

# Make this directory if it doesn't exist
os.makedirs(simulation_out_path, exist_ok=True)

In [ ]:
# Now finish setting up the batch runner
map_ = {
    'lv.t_tr' : ['lv.t_tr',],
    'la.t_tr' : ['la.t_tr',],
    'la.delay' : ['la.delay',],
    'lv.tau' : ['lv.tau',],
    'la.tau' : ['la.tau',],
    'lv.t_max' : ['lv.t_max',],
    'la.t_max' : ['la.t_max',],
}

# Map the sample timings
br.map_sample_timings(
    ref_time=1000., # double check if 1000 or 1
    map=map_
    )

In [ ]:
# Map the vessel volumes
br.map_vessel_volume()


# Save the samples to a CSV file
br.samples.to_csv(os.path.join(simulation_out_path,
                                f'input_samples_{n_samples}.csv'),
                                index=False)

# Set up the model with the parameters and time setup
br.setup_model(model=NaghaviModel, po=NaghaviModelParameters,
                time_setup=TEMPLATE_TIME_SETUP_DICT)

In [ ]:
simulations, bool_indices = simulate_data(
    batch_runner=br,
    simulation_out_path=simulation_out_path
)

In [ ]:
summary_df = calculate_output_features(
        simulations=simulations,
        simulation_out_path=simulation_out_path)